In [67]:
import time

from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(message)s',level=logging.DEBUG)

from elasticsearch import Elasticsearch
from clickhouse_driver import Client

class TestRunner:
    def __init__(self, name, config):
        self.name = name
        self.config = config
    
    def run(self, case):
        pass

class ElasticsearchRuner(TestRunner):
    def __init__(self, name, config):
        TestRunner.__init__(self, name, config)
        self.client = Elasticsearch([self.config['cluster']])
        self.indices = self.config['indices']
           
    def run(self, case):
        query = case
        
        logging.info(f'{self.name} run query {case}')
        start = time.time()
        response = self.client.search(index=self.indices, body=query)
        end = time.time()
        logging.info(f'{self.name} run query complete')
        
        result = {}
        result['count'] = response['hits']['total']['value']
        result['response'] = response
        result['elapsed'] = end - start
        
        logging.info(f'query result count {result["count"]} elapsed {result["elapsed"]}')
        return result
        
class ClickhouseRuner(TestRunner):
    def __init__(self, name, config):
        TestRunner.__init__(self, name, config)
        self.client = Client(self.config['cluster'])
    
    def run(self, case):
        query = case
        logging.info(f'{self.name} run query {case}')
        start = time.time()
        response = self.client.execute(query)
        end = time.time()
        logging.info(f'{self.name} run query complete')
        
        result = {}
        result['count'] = len(response)
        result['response'] = response
        result['elapsed'] = end - start
        
        logging.info(f'query result count {result["count"]} elapsed {result["elapsed"]}')
        return result

In [65]:
clickhouse_config = { "cluster" : "host.docker.internal"} 
clickhouse_runner = ClickhouseRuner('clickhouse', clickhouse_config)

query = 'SELECT * FROM syslog'
clickhouse_runner.run(query)

2021-02-25 18:24:15,934 clickhouse run query SELECT * FROM syslog
2021-02-25 18:24:15,971 clickhouse run query complete
2021-02-25 18:24:15,972 query result count 2000 elapsed 0.0353543758392334


{'count': 2000,
 'response': [('meln1ks',
   'random.us',
   "A bug was encountered but not in Vector, which doesn't have bugs",
   'ID849',
   '7516',
   49,
   "<49>2 2021-02-24T23:46:04.415Z random.us meln1ks 7516 ID849 - A bug was encountered but not in Vector, which doesn't have bugs",
   datetime.datetime(2021, 2, 24, 23, 46, 4, tzinfo=<UTC>),
   2),
  ('Karimmove',
   'we.net',
   "You're not gonna believe what just happened",
   'ID997',
   '8646',
   113,
   "<113>2 2021-02-24T23:46:04.425Z we.net Karimmove 8646 ID997 - You're not gonna believe what just happened",
   datetime.datetime(2021, 2, 24, 23, 46, 4, tzinfo=<UTC>),
   2),
  ('devankoshal',
   'for.org',
   "Great Scott! We're never gonna reach 88 mph with the flux capacitor in its current state!",
   'ID216',
   '3366',
   154,
   "<154>1 2021-02-24T23:46:04.435Z for.org devankoshal 3366 ID216 - Great Scott! We're never gonna reach 88 mph with the flux capacitor in its current state!",
   datetime.datetime(2021, 2, 24

In [66]:
es_config = { "cluster" : "host.docker.internal", "indices" : "syslog-2021-02-24,syslog-2021-02-25"} 
es_runner = ElasticsearchRuner('es', es_config)

query = {
    "query": {
        "match_all": {}
    }
}
es_runner.run(query)

2021-02-25 18:24:22,393 es run query {'query': {'match_all': {}}}
2021-02-25 18:24:22,404 es run query complete
2021-02-25 18:24:22,405 query result count 2000 elapsed 0.009815454483032227


{'count': 2000,
 'response': {'took': 1,
  'timed_out': False,
  '_shards': {'total': 2, 'successful': 2, 'skipped': 0, 'failed': 0},
  'hits': {'total': {'value': 2000, 'relation': 'eq'},
   'max_score': 1.0,
   'hits': [{'_index': 'syslog-2021-02-24',
     '_type': '_doc',
     '_id': 'iNVu1ncBIYMyHW4dba-j',
     '_score': 1.0,
     '_source': {'application': 'ahmadajmi',
      'hostname': 'random.net',
      'message': 'Pretty pretty pretty good',
      'mid': 'ID3',
      'pid': '4895',
      'priority': 117,
      'raw': '<117>2 2021-02-24T23:44:56.160Z random.net ahmadajmi 4895 ID3 - Pretty pretty pretty good',
      'timestamp': '2021-02-24T23:44:56.160Z',
      'version': 2}},
    {'_index': 'syslog-2021-02-24',
     '_type': '_doc',
     '_id': 'idVu1ncBIYMyHW4dba-j',
     '_score': 1.0,
     '_source': {'application': 'devankoshal',
      'hostname': 'make.de',
      'message': "You're not gonna believe what just happened",
      'mid': 'ID51',
      'pid': '9927',
      'pri

In [75]:
import json

class TestPlan:
    def __init__(self, planfile):
        with open(planfile) as json_file:
            self.plan = json.load(json_file)
            self.targets = []
            for target in self.plan['targets']:
                if target['type'] == 'elasticsearch':
                    runner = ElasticsearchRuner(target['name'], target['config'])
                    self.targets.append(runner)
                elif target['type'] == 'clickhouse':
                    runner = ClickhouseRuner(target['name'], target['config'])
                    self.targets.append(runner)
    
    def run(self):
        reports = []
        for testcase in self.plan['testcases']:
            name, description, cases = testcase
            report = {}
            report['name'] = name
            report['description'] = description
            report['result'] = []
            logging.info(f'run testcase {name} {description}')
            tests = list(zip(self.targets, cases))
            for test in tests:
                runner, query = test
                result = {}
                result['target'] = runner.name
                result['case'] = str(query)
                
                elapsed = 0
                for i in range(self.plan['runs']):
                    run_result = runner.run(query)
                    elapsed += run_result['elapsed']
                    
                result['elapsed'] = elapsed
                report['result'].append(result)
                
            logging.info(f'run testcase {name} complete')
            reports.append(report)
        return reports


testplan = TestPlan("testplan.json")
reports = testplan.run()

2021-02-25 18:33:50,255 run testcase case1 query all
2021-02-25 18:33:50,257 es1 run query {'query': {'match_all': {}}}
2021-02-25 18:33:50,274 es1 run query complete
2021-02-25 18:33:50,276 query result count 2000 elapsed 0.014372110366821289
2021-02-25 18:33:50,278 es1 run query {'query': {'match_all': {}}}
2021-02-25 18:33:50,288 es1 run query complete
2021-02-25 18:33:50,290 query result count 2000 elapsed 0.008500814437866211
2021-02-25 18:33:50,292 es1 run query {'query': {'match_all': {}}}
2021-02-25 18:33:50,305 es1 run query complete
2021-02-25 18:33:50,307 query result count 2000 elapsed 0.010248661041259766
2021-02-25 18:33:50,309 es1 run query {'query': {'match_all': {}}}
2021-02-25 18:33:50,316 es1 run query complete
2021-02-25 18:33:50,318 query result count 2000 elapsed 0.004602670669555664
2021-02-25 18:33:50,319 es1 run query {'query': {'match_all': {}}}
2021-02-25 18:33:50,328 es1 run query complete
2021-02-25 18:33:50,330 query result count 2000 elapsed 0.00734448432

2021-02-25 18:33:50,947 es1 run query {'query': {'match_all': {}}}
2021-02-25 18:33:50,959 es1 run query complete
2021-02-25 18:33:50,962 query result count 2000 elapsed 0.008811235427856445
2021-02-25 18:33:50,964 es1 run query {'query': {'match_all': {}}}
2021-02-25 18:33:50,974 es1 run query complete
2021-02-25 18:33:50,977 query result count 2000 elapsed 0.008055686950683594
2021-02-25 18:33:50,978 es1 run query {'query': {'match_all': {}}}
2021-02-25 18:33:50,987 es1 run query complete
2021-02-25 18:33:50,989 query result count 2000 elapsed 0.006334543228149414
2021-02-25 18:33:50,991 es1 run query {'query': {'match_all': {}}}
2021-02-25 18:33:50,998 es1 run query complete
2021-02-25 18:33:50,999 query result count 2000 elapsed 0.005852460861206055
2021-02-25 18:33:51,000 es1 run query {'query': {'match_all': {}}}
2021-02-25 18:33:51,012 es1 run query complete
2021-02-25 18:33:51,014 query result count 2000 elapsed 0.011014699935913086
2021-02-25 18:33:51,017 es1 run query {'query

2021-02-25 18:33:51,517 es1 run query {'query': {'match_all': {}}}
2021-02-25 18:33:51,532 es1 run query complete
2021-02-25 18:33:51,539 query result count 2000 elapsed 0.013809442520141602
2021-02-25 18:33:51,543 es1 run query {'query': {'match_all': {}}}
2021-02-25 18:33:51,551 es1 run query complete
2021-02-25 18:33:51,553 query result count 2000 elapsed 0.005274295806884766
2021-02-25 18:33:51,555 es1 run query {'query': {'match_all': {}}}
2021-02-25 18:33:51,562 es1 run query complete
2021-02-25 18:33:51,564 query result count 2000 elapsed 0.005836009979248047
2021-02-25 18:33:51,565 es1 run query {'query': {'match_all': {}}}
2021-02-25 18:33:51,573 es1 run query complete
2021-02-25 18:33:51,575 query result count 2000 elapsed 0.0075223445892333984
2021-02-25 18:33:51,577 es1 run query {'query': {'match_all': {}}}
2021-02-25 18:33:51,585 es1 run query complete
2021-02-25 18:33:51,587 query result count 2000 elapsed 0.005248308181762695
2021-02-25 18:33:51,588 es1 run query {'quer

2021-02-25 18:33:52,394 query result count 2000 elapsed 0.018829345703125
2021-02-25 18:33:52,395 clickhouse1 run query SELECT raw FROM syslog
2021-02-25 18:33:52,416 clickhouse1 run query complete
2021-02-25 18:33:52,418 query result count 2000 elapsed 0.019451379776000977
2021-02-25 18:33:52,420 clickhouse1 run query SELECT raw FROM syslog
2021-02-25 18:33:52,436 clickhouse1 run query complete
2021-02-25 18:33:52,438 query result count 2000 elapsed 0.014980316162109375
2021-02-25 18:33:52,443 clickhouse1 run query SELECT raw FROM syslog
2021-02-25 18:33:52,513 clickhouse1 run query complete
2021-02-25 18:33:52,515 query result count 2000 elapsed 0.06811094284057617
2021-02-25 18:33:52,516 clickhouse1 run query SELECT raw FROM syslog
2021-02-25 18:33:52,538 clickhouse1 run query complete
2021-02-25 18:33:52,540 query result count 2000 elapsed 0.018648386001586914
2021-02-25 18:33:52,541 clickhouse1 run query SELECT raw FROM syslog
2021-02-25 18:33:52,558 clickhouse1 run query complete

2021-02-25 18:33:53,303 query result count 2000 elapsed 0.01655435562133789
2021-02-25 18:33:53,304 clickhouse1 run query SELECT raw FROM syslog
2021-02-25 18:33:53,328 clickhouse1 run query complete
2021-02-25 18:33:53,330 query result count 2000 elapsed 0.022090911865234375
2021-02-25 18:33:53,331 clickhouse1 run query SELECT raw FROM syslog
2021-02-25 18:33:53,357 clickhouse1 run query complete
2021-02-25 18:33:53,359 query result count 2000 elapsed 0.025041818618774414
2021-02-25 18:33:53,360 clickhouse1 run query SELECT raw FROM syslog
2021-02-25 18:33:53,384 clickhouse1 run query complete
2021-02-25 18:33:53,386 query result count 2000 elapsed 0.0228884220123291
2021-02-25 18:33:53,387 clickhouse1 run query SELECT raw FROM syslog
2021-02-25 18:33:53,403 clickhouse1 run query complete
2021-02-25 18:33:53,405 query result count 2000 elapsed 0.015581846237182617
2021-02-25 18:33:53,407 clickhouse1 run query SELECT raw FROM syslog
2021-02-25 18:33:53,423 clickhouse1 run query complet

2021-02-25 18:33:54,132 es1 run query {'query': {'match': {'hostname': 'for.org'}}}
2021-02-25 18:33:54,144 es1 run query complete
2021-02-25 18:33:54,145 query result count 55 elapsed 0.010818719863891602
2021-02-25 18:33:54,148 es1 run query {'query': {'match': {'hostname': 'for.org'}}}
2021-02-25 18:33:54,157 es1 run query complete
2021-02-25 18:33:54,162 query result count 55 elapsed 0.007296562194824219
2021-02-25 18:33:54,166 es1 run query {'query': {'match': {'hostname': 'for.org'}}}
2021-02-25 18:33:54,178 es1 run query complete
2021-02-25 18:33:54,180 query result count 55 elapsed 0.008223772048950195
2021-02-25 18:33:54,182 es1 run query {'query': {'match': {'hostname': 'for.org'}}}
2021-02-25 18:33:54,188 es1 run query complete
2021-02-25 18:33:54,189 query result count 55 elapsed 0.0051746368408203125
2021-02-25 18:33:54,190 es1 run query {'query': {'match': {'hostname': 'for.org'}}}
2021-02-25 18:33:54,198 es1 run query complete
2021-02-25 18:33:54,200 query result count 5

2021-02-25 18:33:54,526 es1 run query {'query': {'match': {'hostname': 'for.org'}}}
2021-02-25 18:33:54,534 es1 run query complete
2021-02-25 18:33:54,535 query result count 55 elapsed 0.006652116775512695
2021-02-25 18:33:54,536 es1 run query {'query': {'match': {'hostname': 'for.org'}}}
2021-02-25 18:33:54,543 es1 run query complete
2021-02-25 18:33:54,543 query result count 55 elapsed 0.005545854568481445
2021-02-25 18:33:54,544 es1 run query {'query': {'match': {'hostname': 'for.org'}}}
2021-02-25 18:33:54,552 es1 run query complete
2021-02-25 18:33:54,553 query result count 55 elapsed 0.006623744964599609
2021-02-25 18:33:54,554 es1 run query {'query': {'match': {'hostname': 'for.org'}}}
2021-02-25 18:33:54,562 es1 run query complete
2021-02-25 18:33:54,567 query result count 55 elapsed 0.005877017974853516
2021-02-25 18:33:54,568 es1 run query {'query': {'match': {'hostname': 'for.org'}}}
2021-02-25 18:33:54,576 es1 run query complete
2021-02-25 18:33:54,577 query result count 55

2021-02-25 18:33:54,913 es1 run query {'query': {'match': {'hostname': 'for.org'}}}
2021-02-25 18:33:54,919 es1 run query complete
2021-02-25 18:33:54,922 query result count 55 elapsed 0.005201101303100586
2021-02-25 18:33:54,923 es1 run query {'query': {'match': {'hostname': 'for.org'}}}
2021-02-25 18:33:54,932 es1 run query complete
2021-02-25 18:33:54,934 query result count 55 elapsed 0.006247043609619141
2021-02-25 18:33:54,935 es1 run query {'query': {'match': {'hostname': 'for.org'}}}
2021-02-25 18:33:54,941 es1 run query complete
2021-02-25 18:33:54,942 query result count 55 elapsed 0.005169868469238281
2021-02-25 18:33:54,944 es1 run query {'query': {'match': {'hostname': 'for.org'}}}
2021-02-25 18:33:54,950 es1 run query complete
2021-02-25 18:33:54,951 query result count 55 elapsed 0.005788087844848633
2021-02-25 18:33:54,953 es1 run query {'query': {'match': {'hostname': 'for.org'}}}
2021-02-25 18:33:54,959 es1 run query complete
2021-02-25 18:33:54,960 query result count 55

2021-02-25 18:33:55,459 query result count 55 elapsed 0.01313924789428711
2021-02-25 18:33:55,461 clickhouse1 run query SELECT raw FROM syslog WHERE hostname='for.org'
2021-02-25 18:33:55,475 clickhouse1 run query complete
2021-02-25 18:33:55,476 query result count 55 elapsed 0.012624025344848633
2021-02-25 18:33:55,477 clickhouse1 run query SELECT raw FROM syslog WHERE hostname='for.org'
2021-02-25 18:33:55,492 clickhouse1 run query complete
2021-02-25 18:33:55,493 query result count 55 elapsed 0.013423681259155273
2021-02-25 18:33:55,494 clickhouse1 run query SELECT raw FROM syslog WHERE hostname='for.org'
2021-02-25 18:33:55,505 clickhouse1 run query complete
2021-02-25 18:33:55,506 query result count 55 elapsed 0.009723663330078125
2021-02-25 18:33:55,507 clickhouse1 run query SELECT raw FROM syslog WHERE hostname='for.org'
2021-02-25 18:33:55,520 clickhouse1 run query complete
2021-02-25 18:33:55,522 query result count 55 elapsed 0.011870145797729492
2021-02-25 18:33:55,523 clickh

2021-02-25 18:33:56,068 clickhouse1 run query complete
2021-02-25 18:33:56,069 query result count 55 elapsed 0.009931325912475586
2021-02-25 18:33:56,070 clickhouse1 run query SELECT raw FROM syslog WHERE hostname='for.org'
2021-02-25 18:33:56,086 clickhouse1 run query complete
2021-02-25 18:33:56,087 query result count 55 elapsed 0.014201879501342773
2021-02-25 18:33:56,088 clickhouse1 run query SELECT raw FROM syslog WHERE hostname='for.org'
2021-02-25 18:33:56,099 clickhouse1 run query complete
2021-02-25 18:33:56,103 query result count 55 elapsed 0.010640382766723633
2021-02-25 18:33:56,104 clickhouse1 run query SELECT raw FROM syslog WHERE hostname='for.org'
2021-02-25 18:33:56,117 clickhouse1 run query complete
2021-02-25 18:33:56,118 query result count 55 elapsed 0.01070094108581543
2021-02-25 18:33:56,118 clickhouse1 run query SELECT raw FROM syslog WHERE hostname='for.org'
2021-02-25 18:33:56,130 clickhouse1 run query complete
2021-02-25 18:33:56,131 query result count 55 elap

In [76]:
reports

[{'name': 'case1',
  'description': 'query all',
  'result': [{'target': 'es1',
    'case': "{'query': {'match_all': {}}}",
    'elapsed': 0.7781581878662109},
   {'target': 'clickhouse1',
    'case': 'SELECT raw FROM syslog',
    'elapsed': 1.7848196029663086}]},
 {'name': 'case2',
  'description': 'query all',
  'result': [{'target': 'es1',
    'case': "{'query': {'match': {'hostname': 'for.org'}}}",
    'elapsed': 0.617067813873291},
   {'target': 'clickhouse1',
    'case': "SELECT raw FROM syslog WHERE hostname='for.org'",
    'elapsed': 1.2508032321929932}]}]